In [88]:
import torch
from transformers import BertTokenizer, BertModel, GPT2Tokenizer, GPT2Model
import pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split
import pandas as pd
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

In [89]:


# Load and preprocess data
def load_data(file_path):
    data = pd.read_csv(file_path)
    texts = data['text'].tolist()
    scores = data['score'].tolist()
    return texts, scores

In [90]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [91]:



# Tokenize and create BERT embeddings
def create_bert_embeddings(texts):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained(
        'bert-base-uncased',
        output_hidden_states = True,
    )


    embeddings = []
    num_layers = 10
    for text in tqdm(texts, desc="Creating BERT Embeddings"):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        outputs = model(**inputs)
        
        
        hidden_states = torch.stack(outputs.hidden_states[-num_layers:])
#         embeddings.append(outputs.pooler_output.detach().numpy().flatten())
        embedding = torch.cat([hidden_states[i] for i in range(num_layers)], dim=-1)
        embedding = torch.mean(embedding, dim=1).squeeze().detach().numpy()
        embeddings.append(embedding)


    return embeddings



def create_gpt2_embeddings(texts):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    model = GPT2Model.from_pretrained('gpt2')
    model = model.to(device)

    embeddings = []
    for text in tqdm(texts, desc="Creating GPT-2 Embeddings"):
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
        inputs = inputs.to(device)
        
        with torch.no_grad():
            outputs = model(**inputs)
            last_hidden_states = outputs.last_hidden_state

        # Use the mean of the last hidden states as the embedding
        embedding = torch.mean(last_hidden_states, dim=1).squeeze().detach().cpu().numpy()
        embeddings.append(embedding)

    return embeddings


In [110]:

# Define the MLP model
class MLPModel(nn.Module):
    def __init__(self):
        super(MLPModel, self).__init__()
        self.l1 = nn.Sequential(
            
            # BERT 10
            # nn.Linear(5632, 4096),
            # nn.ReLU(),
            nn.Linear(5632, 1024),
            nn.ReLU(),
            # nn.Linear(2048, 1024),
            # nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),   
            nn.ReLU(),
            nn.Linear(256, 128),   
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )


    def forward(self, x):
        x = self.l1(x)
        return x

In [111]:

# Train MLP model
def train_mlp_model(train_loader, val_loader, model, criterion, optimizer, num_epochs=5):
    model.to(device)

    for epoch in tqdm(range(num_epochs)):
        model.train()
        # for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Training"):
        train_preds = []
        train_labels = []
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, labels.unsqueeze(1).float())
            loss.backward()
            optimizer.step()

        
            train_preds.extend(outputs.detach().cpu().numpy())
            train_labels.extend(labels.detach().cpu().numpy())

        train_rmse = mean_squared_error(train_labels, train_preds, squared=False)
        print(f"Epoch {epoch + 1}/{num_epochs}, Test RMSE: {train_rmse:.4f}")

        
        model.eval()
        val_preds = []
        val_labels = []
        with torch.no_grad():
            # for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch + 1}/{num_epochs} - Validation"):
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                val_preds.extend(outputs.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_rmse = mean_squared_error(val_labels, val_preds, squared=False)
        print(f"Epoch {epoch + 1}/{num_epochs}, Validation RMSE: {val_rmse:.4f}")


In [112]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

def get_embeddings(texts, model_name):
    # Load pre-trained model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=514)
    model = AutoModel.from_pretrained(
        model_name,
    ).to(device)

    embeddings = []

    for text in tqdm(texts):
        # Tokenize input text and obtain embeddings
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        # Extract the embeddings from the last layer
        last_hidden_states = outputs.last_hidden_state
        # Average pooling to get a fixed-size embedding for the entire text
        avg_pooling = torch.mean(last_hidden_states, dim=1)
        # Convert tensor to numpy array
        avg_pooling = avg_pooling.cpu().numpy()
        # Append the embedding to the list
        embeddings.append(avg_pooling)

    return np.concatenate(embeddings, axis=0)

models = [
    'xlm-mlm-enfr-1024',
    'distilbert-base-cased',
    'bert-base-uncased',
    'roberta-base',
    
    
    'bert-base-cased',
    'xlm-roberta-base',
    'xlnet-base-cased',    

# 'cardiffnlp/twitter-roberta-base-sentiment',
#     'ctrl',
#     'transfo-xl-wt103',
#     'openai-gpt',
#     'gpt2'
]


def get_all_embeddings(texts):
    all_embeddings = []

    for model_name in models:
        print(f"calculating embeddings for {model_name}")
        embeddings = get_embeddings(texts, model_name)
        all_embeddings.append(embeddings)

    return np.concatenate(all_embeddings, axis=1)




In [95]:

# Load and preprocess training data
train_texts, train_scores = load_data('train.csv')


# Split data into training and validation sets
train_texts, val_texts, train_scores, val_scores = train_test_split(
    train_texts, train_scores, test_size=0.2, random_state=42
)


embedding_function = get_all_embeddings

# Create BERT embeddings
train_embeddings = embedding_function(train_texts)
val_embeddings = embedding_function(val_texts)

calculating embeddings for xlm-mlm-enfr-1024


100%|██████████████████████████████████████████████████████████████████| 2400/2400 [00:39<00:00, 60.22it/s]


calculating embeddings for distilbert-base-cased


100%|█████████████████████████████████████████████████████████████████| 2400/2400 [00:19<00:00, 124.79it/s]


calculating embeddings for bert-base-uncased


...

100%|████████████████████████████████████████████████████████████████████| 600/600 [00:09<00:00, 65.34it/s]


calculating embeddings for xlm-roberta-base


100%|████████████████████████████████████████████████████████████████████| 600/600 [00:09<00:00, 65.22it/s]


calculating embeddings for xlnet-base-cased


100%|████████████████████████████████████████████████████████████████████| 600/600 [00:17<00:00, 34.33it/s]


In [102]:

# Convert data to PyTorch tensors
train_data = TensorDataset(torch.tensor(train_embeddings), torch.tensor(train_scores))
val_data = TensorDataset(torch.tensor(val_embeddings), torch.tensor(val_scores))


In [103]:
len(train_embeddings[0])

5632

In [128]:


# Create DataLoader
batch_size = 1024
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

# model = MLPModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.5e-6)

num_epochs = 1000
train_mlp_model(train_loader, val_loader, model, criterion, optimizer, num_epochs=num_epochs)

  0%|▏                                                                    | 3/1000 [00:00<00:48, 20.53it/s]

Epoch 1/1000, Test RMSE: 0.2912
Epoch 1/1000, Validation RMSE: 0.3577
Epoch 2/1000, Test RMSE: 0.2911
Epoch 2/1000, Validation RMSE: 0.3577
Epoch 3/1000, Test RMSE: 0.2911
Epoch 3/1000, Validation RMSE: 0.3576
Epoch 4/1000, Test RMSE: 0.2911
Epoch 4/1000, Validation RMSE: 0.3576
Epoch 5/1000, Test RMSE: 0.2910
Epoch 5/1000, Validation RMSE: 0.3576


  1%|▌                                                                    | 9/1000 [00:00<00:46, 21.09it/s]

Epoch 6/1000, Test RMSE: 0.2911
Epoch 6/1000, Validation RMSE: 0.3576
Epoch 7/1000, Test RMSE: 0.2910
Epoch 7/1000, Validation RMSE: 0.3576
Epoch 8/1000, Test RMSE: 0.2910
Epoch 8/1000, Validation RMSE: 0.3576
Epoch 9/1000, Test RMSE: 0.2910
Epoch 9/1000, Validation RMSE: 0.3576
Epoch 10/1000, Test RMSE: 0.2910
Epoch 10/1000, Validation RMSE: 0.3576


  2%|█                                                                   | 15/1000 [00:00<00:46, 21.19it/s]

...

Epoch 990/1000, Test RMSE: 0.2831
Epoch 990/1000, Validation RMSE: 0.3578
Epoch 991/1000, Test RMSE: 0.2831
Epoch 991/1000, Validation RMSE: 0.3578
Epoch 992/1000, Test RMSE: 0.2831
Epoch 992/1000, Validation RMSE: 0.3578
Epoch 993/1000, Test RMSE: 0.2831
Epoch 993/1000, Validation RMSE: 0.3578
Epoch 994/1000, Test RMSE: 0.2832
Epoch 994/1000, Validation RMSE: 0.3578


100%|██████████████████████████████████████████████████████████████████▉| 999/1000 [00:49<00:00, 21.04it/s]

Epoch 995/1000, Test RMSE: 0.2831
Epoch 995/1000, Validation RMSE: 0.3578
Epoch 996/1000, Test RMSE: 0.2831
Epoch 996/1000, Validation RMSE: 0.3578
Epoch 997/1000, Test RMSE: 0.2831
Epoch 997/1000, Validation RMSE: 0.3578
Epoch 998/1000, Test RMSE: 0.2831
Epoch 998/1000, Validation RMSE: 0.3578
Epoch 999/1000, Test RMSE: 0.2830
Epoch 999/1000, Validation RMSE: 0.3578


100%|██████████████████████████████████████████████████████████████████| 1000/1000 [00:49<00:00, 20.10it/s]

Epoch 1000/1000, Test RMSE: 0.2831
Epoch 1000/1000, Validation RMSE: 0.3578


In [129]:
test_texts, _ = load_data('sample_submission.csv')
test_embeddings = embedding_function(test_texts)


test_data = TensorDataset(torch.tensor(test_embeddings))
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

calculating embeddings for xlm-mlm-enfr-1024


100%|████████████████████████████████████████████████████████████████████| 500/500 [00:08<00:00, 60.78it/s]


calculating embeddings for distilbert-base-cased


100%|███████████████████████████████████████████████████████████████████| 500/500 [00:03<00:00, 128.05it/s]


calculating embeddings for bert-base-uncased


...

100%|████████████████████████████████████████████████████████████████████| 500/500 [00:07<00:00, 66.92it/s]


calculating embeddings for xlm-roberta-base


100%|████████████████████████████████████████████████████████████████████| 500/500 [00:07<00:00, 65.98it/s]


calculating embeddings for xlnet-base-cased


100%|████████████████████████████████████████████████████████████████████| 500/500 [00:14<00:00, 34.80it/s]


In [132]:

model.eval()
test_preds = []
with torch.no_grad():
    for inputs in tqdm(test_loader, desc="Generating Test Predictions"):
        inputs = inputs[0].to(device)

        outputs = model(inputs)
        test_preds.extend(outputs.cpu().numpy())


test_preds = [x[0] for x  in test_preds]

        
# Write predictions to the output CSV file
output_df = pd.DataFrame({'text': test_texts, 'score': test_preds})
output_df.to_csv('combination.csv', index=False)


Generating Test Predictions: 100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 153.09it/s]
